# Introduction

---

## 1. Introducing gated convolutions
In the first experiment with a basic convolutional neural network (CNN), we observed that the model produced various types of visual artifacts, from discolouration to patterns that resembled the shape of the patch, and we hypothesized that it arose from convolutions operating on the hole values (set to 0, by default).

Following the work of [Liu et al. (2018)](https://arxiv.org/pdf/1804.07723.pdf)and [Yu et. al (2018)](https://arxiv.org/abs/1806.03589), we implement gated convolutions as a simple yet flexible way to ensure that "invalid" pixels are not used in the image generation process in a learnable fashion.

As a means of providing more information to the model, we also provide the mask as an input channel.

---

## 2. Modified U-Net

### 2.1 Choice of U-Net architecture

In our earlier experiment with a CNN, we noted that skip connections would likely help with faster training. We also noted that maintaining activation maps of equal dimensions at all layers was computationally expensive. To that end, a model design with skip connections, a downsampling then upsampling operation with the ability to reconstruct pixel-level details would be ideal. The U-Net architecture proposed by [Ronneberger et al. (2015)](https://arxiv.org/abs/1505.04597) suits this.

### 2.2 Introducing residual skips

As argued by [He et al. (2015)](https://arxiv.org/abs/1512.03385), residual skips allow for the practical training of deeper neural networks with higher representational power, as the improved gradient flow alleviates the vanishing gradient problem and residual skips tend toward the identity function, rather than the zero function. 

To increase the receptive field of each masked pixel in the output, we need to deepen the network, and so introduce residual skips as a practical matter. 

### 2.3 Downsampling by convolutions, rather than pooling

In the original paper, the U-Net model was designed for image segmentation and uses max-pooling for downsampling. Owing to the translation and (somewhat) rotation invariance of max pooling, this operator works well for segmentation. But the loss of localization information likely leads to lossy downsampling in the application to image generation for inpainting, so instead, we carry out downsampling by convolutions with larger strides.

So, we use a U-Net like architecture, with the added difference of residual skips at every block to improve gradient flow, as well as downsampling using convolutions, rather than pooling to avoid lossy downsampling (since our goal is to reconstruct the image).

### 2.4. Upsampling by convolutions and pixel shuffling

There is [evidence](https://distill.pub/2016/deconv-checkerboard/) to suggest that the transposed convolution operation as a manner of learnable upsampling tends to generate checkboard-like artefacts, due to overlapping points in the application of the convolution filter at the low-resolution feature maps. 

An alternative, thus, is to use nearest-neighbour interpolation paired with convolutions to upsample images. However, [Shi et al. (2017)](https://arxiv.org/ftp/arxiv/papers/1609/1609.07009.pdf) argue that carrying out upsampling by what they call "pixel shuffling" has higher representational power for the same computational complexity. The operation they propose works by using convolutions to generate *n x r<sup>2</sup> x h x w* feature maps from an input with *n* feature maps, and then "shuffling" these feature maps together to get an output of *n x (hr) x (wr)*. The intuition behind its greater representational power given the same time complexity, is that the convolutions are operating in the lower-resolution spce, rather than the higher-resolution space after upsampling - so more parameters can be learnt. 

---

## 3. A segmented loss function

In the basic CNN model, we trained the model with L1 loss, with the intuition that it would aid in high-accuracy pixel contruction. 

We observed empirically that the masked areas were much smaller than the overall image, and the error signal would probably not encourage the careful reconstruction of the masked regions. So instead, we can separate these two losess - one for the masked region, one for the unmasked region, and then weight their losses as a hyperparameter.


# Initialization

## Imports

In [ ]:
# functional
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import time

# visualization
import matplotlib.pyplot as plt

# images
from skimage import io
from skimage import transform

# metrics
!pip install torchmetrics
!pip install lpips
import torchmetrics


## Get files

In [ ]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive')

# make a local data directory
!rm -r /content/data
!mkdir /content/data
!mkdir /content/data/images
!mkdir /content/data/preprocessed


In [ ]:
dataset_ids = {
    'frogs': '1hq3Bjd1tapiVt_qmqb6uP8rA5ECutXCE',
    'frog_indices' : '1Td2csrwues_ZSqDCXSIBLLx-Ab7Z0KIJ'
}

# get images
file_id = dataset_ids['frogs']
file_name = '/content/data/frogs_15k_renamed.zip'
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={file_id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={file_id}" -O {file_name} && rm -rf /tmp/cookies.txt

# unzip file
!unzip /content/data/frogs_15k_renamed.zip -d /content/data/images

# get index file
file_id = dataset_ids['frog_indices']
file_name = '/content/data/frogs_wanted_indices_url.txt'
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={file_id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={file_id}" -O {file_name} && rm -rf /tmp/cookies.txt


## Import custom methods

In [ ]:
# dataset and training functions
%cd /content/drive/MyDrive/CS4243\ Project
from dataset_util.AnimalDataset import AnimalDataset
from train_util.train_util import sample_batch, visualize_results

# Model

## Define model
Here, we define three main classes to encapsulate the logoic - a `GatedConv2d` module as a fundemental module, a `GatedConv2dBlock` class that encapsulates batch normalization and activation, and then then main model.

In [ ]:
class GatedConv2d(nn.Module):

    """
    This class implements a gated convolution. It works by applying a convolution filter 
    to the input feature tensor, then using the sigmoid function to map each score to a 
    pixel validity weight. The weights are then multiplied to the activation map from a 
    separate convolution filter.
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(GatedConv2d, self).__init__()
        self.image_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.gate_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):

        mask = self.sigmoid(self.gate_conv(input_tensor))
        x = self.image_conv(input_tensor)
        x = torch.mul(x, mask) # apply mask

        return x


class GatedConv2dBlock(nn.Module):
    """
    This class encapsulates the following:
    GatedConv2d -> BN -> activation
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, activation = nn.ReLU, p = 0.2):
        super(GatedConv2dBlock, self).__init__()
        self.conv = GatedConv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(num_features = out_channels)
        self.activation = activation()
    
    def forward(self, input_tensor):
        x = self.conv(input_tensor)
        x = self.bn(x)
        x = self.activation(x)

        return x

class GatedConv2DPixelShuffleUpsample(nn.Module):
    """
    This class encapsulates a convolution layer using GatedConv
    and PixelShuffling to upsample. Returns same number of channels, with
    output spatial dimensions scaled.
    """

    def __init__(self, in_channels, upscale_factor = 2):

        super(GatedConv2DPixelShuffleUpsample, self).__init__()
        self.conv = GatedConv2dBlock(in_channels, scale_factor ** 2 * in_channels, kernel_size = 3, stride = 1, padding = 'same')
        self.upsample = nn.PixelShuffle(upscale_factor = upscale_factor)
    
    def forward(self, input_tensor):
        
        x = self.conv(input_tensor)
        return self.upsample(x)


class GatedConvUNet(nn.Module):
    """
    This class encapsulates a UNet-like model with gated convolution blocks and residual skips.
    """
    
    def __init__(self, 
                 blocks_per_layer = 2,
                 in_channels = 4, 
                 hidden_channels = 32, 
                 kernel_size = 3, 
                 stride = 1, 
                 padding = 'same',
                 activation = nn.ReLU):

        super(GatedConvUNet, self).__init__()

        # ===== DOWNSAMPLING =====
        
        # first convolution
        self.first = GatedConv2dBlock(in_channels, hidden_channels, kernel_size, stride, padding, activation)

        # convolutions
        self.down_1 = nn.ModuleList(
            [GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])

        self.down_2 = nn.ModuleList(
            [GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])        

        self.down_3 = nn.ModuleList(
            [GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])  
        
        # downsampling by convolution
        self.downtransform_1 = GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size = 4, stride = 2, padding = 1, activation = activation)
        self.downtransform_2 = GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size = 4, stride = 2, padding = 1, activation = activation)
        self.downtransform_3 = GatedConv2dBlock(hidden_channels, 2 * hidden_channels, kernel_size = 4, stride = 2, padding = 1, activation = activation)

        # ===== MIDDLE =====
        
        self.middle = nn.ModuleList(
            [GatedConv2dBlock(2 * hidden_channels, 2 * hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])  
        

        # ===== UPSAMPLING =====
        # upsample and convolution, always doubling
        self.upsample_1 = GatedConv2DPixelShuffleUpsample(in_channels = 2 * hidden_channels, upscale_factor = 2)
        self.uptransform_1 = GatedConv2dBlock(2 * hidden_channels, hidden_channels, kernel_size, stride, padding, activation)
        
        self.upsample_2 = GatedConv2DPixelShuffleUpsample(in_channels = hidden_channels, upscale_factor = 2)
        self.uptransform_2 = GatedConv2dBlock(2 * hidden_channels, hidden_channels, kernel_size, stride, padding, activation)

        self.upsample_3 = GatedConv2DPixelShuffleUpsample(in_channels = hidden_channels, upscale_factor = 2)
        self.uptransform_3 = GatedConv2dBlock(2 * hidden_channels, hidden_channels, kernel_size, stride, padding, activation)

        # convolutions
        self.up_1 = nn.ModuleList(
            [GatedConv2dBlock(2 * hidden_channels, 2 * hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])
        
        self.up_2 = nn.ModuleList(
            [GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])
        
        self.up_3 = nn.ModuleList(
            [GatedConv2dBlock(hidden_channels, hidden_channels, kernel_size, stride, padding, activation) 
             for i in range(blocks_per_layer)])

        # final scores
        self.final = nn.Conv2d(hidden_channels, 3, kernel_size, stride, padding)

    def forward(self, input_tensor):
        
        # ===== DOWNSAMPLING =====

        # first layer
        x1 = self.first(input_tensor)
        for l in self.down_1:
            x1 = x1 + l(x1) # residual skips
        
        # second layer 
        x2 = self.downtransform_1(x1)
        for l in self.down_2:
            x2 = x2 + l(x2)

        # third layer 
        x3 = self.downtransform_2(x2)
        for l in self.down_3:
            x3 = x3 + l(x3)

        # ===== BOTTOM ====
        x = self.downtransform_3(x3)
        for l in self.middle:
            x = x + l(x)

        # ===== UPSAMPLING =====

        # first layer
        x = self.upsample_1(x) # upsample
        x = torch.concat([x, x3], dim = 1) # concat in channel dimension (second)
        x = self.uptransform_1(x)
        for l in self.up_1:
            x = x + l(x)
        
        # second layer 
        x = self.upsample_2(x) # upsample
        x = torch.concat([x, x2], dim = 1) # concat in channel dimension (second)
        x = self.uptransform_2(x)
        for l in self.up_2:
            x = x + l(x)

        # third layer
        x = self.upsample_3(x) # upsample
        x = torch.concat([x, x1], dim = 1) # concat in channel dimension (second)
        x = self.uptransform_3(x)
        for l in self.up_3:
            x = x + l(x)

        # ===== FINAL =====
        x = self.final(x)
        return x 


    def summary(self, verbose = False):
        count = 0
        if verbose:
            print(self)

        for name, params in self.named_parameters():
            num_params = params.flatten().size()[0]
            count += num_params
            if verbose:
                print(f"\nlayer: {name}")
                print(f"number of params: {num_params}")
                print(f"params shape: {params.size()}")

        print(f"model has {count/1e6} million parameters")

## Initialize model
Note that there is a significant difference in the depth (and thus, number of parameters) in the model. There are equivalently 17 layers in this model, relative to the 10 in the original CNN model.

In [ ]:
# model parameters
blocks_per_layer = 1
in_channels = 3
hidden_channels = 32
kernel_size = 3
stride = 1
padding = 'same'
activation = nn.PReLU

# create model
model = GatedConvUNet(blocks_per_layer,
                      in_channels, 
                      hidden_channels, 
                      kernel_size, 
                      stride, 
                      padding,
                      activation)
model.summary(verbose = False)


## Save model

In [ ]:
# go to directory
%cd /content/drive/MyDrive/CS4243\ Project
print(os.getcwd())


In [ ]:
# save path
MODEL_NAME = "GatedConvPixelShuffleUNet_3downup_1blocksperlayer_32hidden_relu"
SAVE_PATH = f"{os.getcwd()}/Naaman/models/{MODEL_NAME}"

# save
torch.save(model.state_dict(), SAVE_PATH)

# sanity check
loaded = GatedConvUNet(blocks_per_layer,
                       in_channels, 
                       hidden_channels, 
                       kernel_size, 
                       stride, 
                       padding,
                       activation)

loaded.load_state_dict(torch.load(SAVE_PATH))
loaded.summary()
del loaded

# Training Details
For training, we define all details here, and wrap it into two main dictionaries, `training_params` and `metrics`.

## Custom loss function

In [ ]:
def mask_segmented_loss(output, target, mask):
    """
    This is a custom functional loss function that considers where the mask is, and
    applies the loss function seperately. 
    """
    
    MASKED_WEIGHT = 10
    UNMASKED_WEIGHT = 1
    LOSS = nn.functional.l1_loss

    # weights by mask, unmasked as 1
    masked_weights = (mask - 1) * -1 * MASKED_WEIGHT # invert the mask in a differentiable way
    unmasked_weights = mask * UNMASKED_WEIGHT
    
    # weighted sum
    loss_masked = LOSS(output, target, reduction = 'none')
    loss_unmasked = LOSS(output, target, reduction = 'none')
    loss = torch.mul(loss_masked, masked_weights) + torch.mul(loss_unmasked, unmasked_weights)
    loss = torch.mean(loss)

    return loss

## Training parameters


In [ ]:
# device
device = 'cuda'  
model = model.to(device)

# define training parameters
num_epochs = 10
batch_size = 64

# optimizer parameters
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, patience = 1, threshold = 1e-6)

# loss function 
loss_function = mask_segmented_loss

# wrap into dictionary
training_params = {
    "num_epochs" : num_epochs,
    "batch_size" : batch_size, 
    "learning_rate": learning_rate,
    "optimizer" : optimizer,
    "scheduler" : scheduler,
    "schedule_every" : 1,
    "loss_function" : loss_function,
    "save_path" : SAVE_PATH
}



## Metrics

In [ ]:
# define metrics
metrics = {
    "Multiscale structural similarity index measure" : torchmetrics.functional.multiscale_structural_similarity_index_measure,
    "Peak SnR" : torchmetrics.functional.peak_signal_noise_ratio
}

## Load Frogs dataset

In [ ]:
dataset = AnimalDataset(index_file_path = "/content/data/frogs_wanted_indices_url.txt",
                        root_dir_path = "/content/data/images/frogs_15k_renamed",
                        local_dir_path = "/content/data/preprocessed",
                        file_prefix = "frog_",
                        image_dimension = 256,
                        concat_mask = True)

dataset.initialize()

# train-test split
VALID_SIZE = 0.2
NUM_WORKERS = 2

indices = torch.arange(len(dataset))
train_indices, validation_indices = train_test_split(indices, test_size = VALID_SIZE)
train_dataset = Subset(dataset, train_indices)
validation_dataset = Subset(dataset, validation_indices)


In [ ]:
sample_batch(dataset, sample_size = 16)
sample_batch(train_dataset, sample_size = 16)
sample_batch(validation_dataset, sample_size = 16)

# Training

## Helper functions

In [ ]:
def train_epoch(model, device, train_dataloader, training_params : dict, metrics : dict):
    """
    This method encapsulates the training of a given model for one epoch. 
    
    @param model : nn.Module                        Model to be trained.
    @param device : str                             Device to be trained on.
    @param train_dataloader : nn.data.DataLoader    DataLoader object to load batches of data.
    @param training_params : dict                   Dictionary object mapping names of 
                                                    training utilities to their respective objects.
                                                    Required are "batch_size", "loss_function", 
                                                    and "optimizer". 
    @param metrics : dict                           Dictionary object mapping names of 
                                                    metrics to a functional method that 
                                                    would compute the metric value.
    """
    
    # ===== INITIALIZE =====
    # constants
    BATCH_SIZE = training_params["batch_size"]
    LOSS_FUNCTION = training_params["loss_function"]
    OPTIMIZER = training_params["optimizer"]
    BATCH_EVALUATE_EVERY = 1

    # model to device
    model = model.to(device)
    model.train()

    # epoch metrics
    running_results = {list(metrics.keys())[i] : 0.0 for i in range(len(metrics)) } 
    running_results["loss"] = 0.0

    # ===== TRAIN EPOCH =====
    num_batches = 0
    for index, batch in enumerate(train_dataloader, 1):

            # ===== INITIALIZE =====
            num_batches += 1

            # input and ground truth
            input_batched = batch["image"]
            ground_truth_batched = batch["reconstructed"]
            mask_batched = batch["mask"]

            # sanity check
            assert input_batched.shape[0] == ground_truth_batched.shape[0]

            # move tensors to device
            input_batched = input_batched.to(device)
            ground_truth_batched = ground_truth_batched.to(device)
            mask_batched = mask_batched.to(device)

            # Set the gradients to zeros
            OPTIMIZER.zero_grad()

            # ===== FORWARD PASS =====
            # reshape to channel first
            input_batched = input_batched.permute(0, 3, 1, 2)
            ground_truth_batched = ground_truth_batched.permute(0, 3, 1, 2)
            mask_batched = mask_batched.permute(0, 3, 1, 2)

            # forward pass
            input_batched.requires_grad_()
            output_batched = model(input_batched)

            # ===== BACKPROP =====

            loss = LOSS_FUNCTION(output_batched, ground_truth_batched, mask_batched)
            loss.backward()
            OPTIMIZER.step()

            # ===== COMPUTE STATISTICS, USING TORCH METRICS =====  
            # for each key, compute, add item to results dictionary
            running_results["loss"] += loss.detach().item()
            for key, func in metrics.items():
                running_results[key] += func(output_batched, ground_truth_batched).detach().item()

            # ===== HOUSEKEEPING =====
            del loss
            del input_batched
            del output_batched

            # print results every some batches
            if num_batches % BATCH_EVALUATE_EVERY == 0: 

                args = ""
                for key, val in running_results.items():
                    args += key + ": " + str(running_results[key]/num_batches) + "   "
                print(f"\r{num_batches}/{len(train_dataloader)}: " + args, end = '', flush = True)

    # normalise numbers by batch
    for key, val in running_results.items():
        running_results[key] /= num_batches

    return running_results



In [ ]:
def evaluate_epoch(model, device, validation_dataloader, training_params : dict, metrics : dict):
    """
    This method encapsulates the evaluation of a given model for one epoch.
    
    @param model : nn.Module                            Model to be trained.
    @param device : str                                 Device to be trained on.
    @param validation_dataloader : nn.data.DataLoader   DataLoader object to load batches of data.
    @param training_params : dict                       Dictionary object mapping names of 
                                                        training utilities to their respective objects.
                                                        Required are "batch_size", "loss_function", 
                                                        and "optimizer". 
    @param metrics : dict                               Dictionary object mapping names of 
                                                        metrics to a functional method that 
                                                        would compute the metric value.
    """
    
    # ===== INITIALIZE =====
    # constants
    BATCH_SIZE = training_params["batch_size"]
    LOSS_FUNCTION = training_params["loss_function"]
    OPTIMIZER = training_params["optimizer"]

    # to device
    model = model.to(device)

    # epoch statistics
    running_results = {list(metrics.keys())[i] : 0.0 for i in range(len(metrics)) } 
    running_results["loss"] = 0.0

    # ===== EVALUATE EPOCH =====

    with torch.no_grad():
        model.eval()
        batches = 0
        for index, batch in enumerate(validation_dataloader, 1):
            
            batches += 1

            # input and ground truth
            input_batched = batch["image"]
            ground_truth_batched = batch["reconstructed"]
            mask_batched = batch["mask"]

            # move tensors to device
            input_batched = input_batched.to(device)
            ground_truth_batched = ground_truth_batched.to(device)
            mask_batched = mask_batched.to(device)

            # reshape to channel first
            input_batched = input_batched.permute(0, 3, 1, 2)
            ground_truth_batched = ground_truth_batched.permute(0, 3, 1, 2)
            mask_batched = mask_batched.permute(0, 3, 1, 2)

            # predict    
            output_batched = model(input_batched)

            # evaluate
            loss = LOSS_FUNCTION(output_batched, ground_truth_batched, mask_batched).detach().item()
            running_results["loss"] += loss
            
            # ===== COMPUTE STATISTICS, USING TORCH METRICS =====
            for key, func in metrics.items():
                running_results[key] += func(output_batched, ground_truth_batched).detach().item()

            args = ""
            for key, val in running_results.items():
                args += key + ": " + str(running_results[key]/batches) + "   "
            print(f"\r{batches}/{len(validation_dataloader)}: " + args, end = '', flush = True)

            # delete to ensure memory footprint
            del loss
            del input_batched
            del output_batched

    # normalise numbers by batch
    for key, val in running_results.items():
        running_results[key] /= batches

    return running_results

    

In [ ]:
def train_evaluate(model, device, train_dataset, validation_dataset, training_params: dict, metrics: dict):

    """
    This method encapsulates the training and evaluation loop of a given model.
    
    @param model : nn.Module                            Model to be trained.
    @param device : str                                 Device to be trained on.
    @param train_dataloader : nn.data.DataLoader        DataLoader object to load batches of data for training.
    @param validation_dataloader : nn.data.DataLoader   DataLoader object to load batches of data for validation.
    @param training_params : dict                       Dictionary object mapping names of 
                                                        training utilities to their respective objects.
                                                        Required are "num_epochs", "batch_size", "loss_function", 
                                                        "scheduler", "save_path" and "optimizer". 
    @param metrics : dict                               Dictionary object mapping names of 
                                                        metrics to a functional method that 
                                                        would compute the metric value.
    """


    # ===== INITIALIZE =====
    # constants
    NUM_EPOCHS = training_params["num_epochs"]
    BATCH_SIZE = training_params["batch_size"]
    SCHEDULER = training_params["scheduler"]
    SAVE_PATH = training_params["save_path"]
    NUM_WORKERS = 2
    START_EPOCH = 0
    SAMPLE_SIZE = 16
    PLOT_EVERY = 1

    # variables
    train_results = {list(metrics.keys())[i] : [] for i in range(len(metrics)) } 
    train_results["loss"] = []
    eval_results = {list(metrics.keys())[i] : [] for i in range(len(metrics)) } 
    eval_results["loss"] = []

    # dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS)
    validation_dataloader = DataLoader(validation_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS)

    # ===== TRAIN =====
    for epoch in range(NUM_EPOCHS):

        start = time.time()

        # train
        print(f"\n===== Epoch: {START_EPOCH + epoch + 1} ===== ")
        num_batches = 0

        # train every epoch
        print("\nTraining ...")
        results = train_epoch(model, device, train_dataloader, training_params, metrics)
        for key, val in results.items():
            train_results[key].append(val)

        # evaluate every epoch
        print("\nEvaluating ...")
        results = evaluate_epoch(model, device, validation_dataloader, training_params, metrics)
        for key, val in results.items():
            eval_results[key].append(val)

        # ===== EPOCH RESULTS =====
        print(f"\nCompleted epoch {START_EPOCH + epoch + 1}! Took {(time.time() - start)/60} min")

        # ===== VISUALIZE =====
        if epoch % PLOT_EVERY == 0:
            print("plotting ...")
            loader = DataLoader(validation_dataset, batch_size = SAMPLE_SIZE, shuffle = True)
            batch = next(iter(loader))

            # predict and plot
            output = model(batch["image"].to(device).permute(0, 3, 1, 2)).detach().cpu().permute(0, 3, 1, 2)
            fig, ax = plt.subplots(3, SAMPLE_SIZE, figsize = (SAMPLE_SIZE * 5, 15, ))
            for i in range(SAMPLE_SIZE):
                image = batch["image"][i]
                reconstructed = batch["reconstructed"][i]
                predicted = output[i]

                if image.shape[-1] > 3: 
                    image = image[:, :, 0:3] # take rgb if more than 3 channels
                    
                ax[0][i].imshow(image)
                ax[1][i].imshow(reconstructed)
                ax[2][i].imshow(predicted)
            plt.savefig(f"{SAVE_PATH}_epoch{epoch + 1}.png")
            print("saved plots!")
            plt.close()

        # ===== HOUSEKEEPING =====

        # scheduler every epoch
        if SCHEDULER is not None:
            SCHEDULER.step(eval_results["loss"][epoch])

        # save save every epoch
        SAVE = f"{SAVE_PATH}_epoch{epoch + 1}.pt"
        torch.save(model.state_dict(), SAVE)
        print("saved model!")

    return train_results, eval_results

## Training

In [ ]:
torch.autograd.set_detect_anomaly(True)

# train
running_train_results, running_eval_results = train_evaluate(model, device, train_dataset, validation_dataset, training_params, metrics)

# Results and analysis

In [ ]:
def visualize_results(model, device, running_train_results: dict, running_eval_results: dict, test_dataset = None, images_only = False):

    # get number of epochs and data points
    NUM_EPOCHS = len(list(running_train_results.values())[0])
    NUM_METRICS = len(list(running_train_results))
    assert list(running_train_results.keys()) == list(running_eval_results.keys())
    
    if not images_only:
        # plot 
        fig, ax = plt.subplots(NUM_METRICS, figsize = (NUM_EPOCHS * 5, NUM_METRICS * 5))
        epochs_axis = [i for i in range(NUM_EPOCHS)]
        index = 0
        for key in list(running_train_results.keys()):
            train = running_train_results[key]
            validation = running_eval_results[key]

            ax[index].plot(epochs_axis, train, label = "train")
            ax[index].plot(epochs_axis, validation, label = "validation")
            ax[index].title.set_text(key)
            index += 1

    if test_dataset != None:
        SAMPLE_SIZE = 8
        loader = DataLoader(test_dataset, batch_size = SAMPLE_SIZE, shuffle = True)
        batch = next(iter(loader))

        # predict
        output = model(batch["image"].to(device).permute(0, 3, 1, 1)).detach().cpu().view(0, 2, 3, 1)

        fig, ax = plt.subplots(3, SAMPLE_SIZE, figsize = (SAMPLE_SIZE * 5, 15, ))
        for i in range(SAMPLE_SIZE):
            image = batch["image"][i]
            reconstructed = batch["reconstructed"][i]
            predicted = output[i]

            if image.shape[-1] > 3: 
                image = image[:, :, 0:3] # take rgb if more than 3 channels
                
            ax[0][i].imshow(image)
            ax[1][i].imshow(reconstructed)
            ax[2][i].imshow(predicted)
        plt.show()
        plt.close()
    

In [ ]:
visualize_results(model, device, running_train_results, running_eval_results, test_dataset = validation_dataset)